In [4]:
### Make CSV of CONUS2 flow at matched gages ###
### DTT, 05/23

# This script is split into two main parts: 1) reading in CONUS2 gridded total (aggregated) daily flow for the full domain and creating a dataframe of CONUS2 flow for cells that have been matched with USGS gages in the `NWM_Gage_Adjustments_final.csv`. 2) matching the gages that both have flow between the PF csv and USGS csv retrieved from hydrodata. 
# Note that flow is converted in this script from daily accumulated flow in [m^3/h] to daily mean flow in cms or [m^3/s].

### Inputs:
# - `NWM_Gage_Adjustments_final.csv` - this can be found on the CONUS2 Dropbox or in /glade/p/univ/ucsm0002/CONUS2/domain_files
# - Daily total streamflow PFCLM outputs as PFBs - processed using `compute_daily_PF_averages.py`
# - USGS daily flow csv - from the hydrodata repository on Verde

### Outputs:
# - CSV of PFCLM daily mean flow (cms) with gage ID, lat/long, and CONUS2 cell location
# - two flow-matched CSVs for PF and USGS flow
# - note that the CSV outputs with 'day 001' which starts at the water year (001 == October 1)***

# Notes:
# - need to fix the no_days, currently this will only be accurate if this is started at the begninning of a water year and need to add in some dictionary or if statement to specify num days in a month or something like that.
# - ***need to change day headings so that they are more descriptive than 'day 001' and have an actual date

import sys
from parflow.tools.io import read_pfb,write_pfb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime


### Directory where PF flow PFBs are saved in ###
## Cheyenne
#directory_out = '/glade/p/univ/ucsm0002/CONUS2/CONUS2.spinup.WY2003/averages'
#organized_dir = '/glade/work/tijerina/PFCONUS2-analysis/scripts/Validation/Streamflow/Organized_Daily_Flow'
## Hydrodata
directory_out = '/hydrodata/temp/CONUS2_transfers/CONUS2/spinup_WY2003/averages'
organized_dir = './Organized_Daily_Flow'

obs_data_file = 'Hydrodata_streamflow_obs_daily_avg_WY2003.csv' #csv of USGS flow from hydrodata
metadata_file = 'Hydrodata_streamflow_metadata_daily_avg_WY2003.csv' #csv of USGS flow from hydrodata

ny = 3256
nx = 4442

# need to change water year and number of days
water_year = 2003
no_days = 364 

# date variables for datetime for wateryear
start_date = datetime.date(water_year, 10, 1)
end_date = datetime.date(water_year, 9, 30)
# daily time step for reading daily files (this could change depending on how you are analyzing)
# delta time
delta = datetime.timedelta(days=1)

In [5]:
# Read observation data and organize
obs_data = pd.read_csv(f'{organized_dir}/{obs_data_file}', index_col=['site_id'])
obs_data = obs_data.drop(columns=['Unnamed: 0'])
# remove sites with less than 365 days of observations
obs_data = obs_data.loc[(obs_data['num_obs']==365)]
# Read metadata and organize
metadata = pd.read_csv(f'{organized_dir}/{metadata_file}', index_col=['site_id'])
metadata = metadata.drop(columns=['Unnamed: 0'])
# also remove the sites with less than 365 obs from the metadata
metadata = metadata[metadata.index.isin(obs_data.index)]

# add number of observations column from the obs_data df
metadata['num_obs'] = obs_data['num_obs']
# remove num_obs from data so we can sum and calc stats
obs_data = obs_data.drop(columns=['num_obs'])

In [6]:
metadata

,site_type,agency,site_name,site_id_agency,site_query_url,date_metadata_last_updated,latitude,longitude,state,huc,...,topo_cd,instruments_cd,construction_dt,inventory_dt,drain_area_va,contrib_drain_area_va,local_time_fg,reliability_cd,project_no,num_obs
site_id,,,,,,,,,,,,,,,,,,,,,
1010000,stream gauge,USGS,"St. John River at Ninemile Bridge, Maine",1010000,https://waterservices.usgs.gov/nwis/site/?form...,2023-05-05,46.700556,-69.715556,ME,1010001,...,NaN,YNNNYNNNNYNNNNNNYNNNNNNNNNNNNN,NaN,NaN,1341.0000,1341.0,Y,NaN,442300100,365
1010070,stream gauge,USGS,"Big Black River near Depot Mtn, Maine",1010070,https://waterservices.usgs.gov/nwis/site/?form...,2023-05-05,46.893889,-69.751667,ME,1010001,...,NaN,YNNNYNYNNYNNNNNNYNNNNNNNNNNNNN,NaN,NaN,171.0000,171.0,Y,NaN,442300100,365
1010500,stream gauge,USGS,"St. John River at Dickey, Maine",1010500,https://waterservices.usgs.gov/nwis/site/?form...,2023-05-05,47.113056,-69.088056,ME,1010001,...,NaN,YNNNYNNNNYNNNYNNYNNNNNNNNNNNNN,NaN,NaN,2680.0000,2680.0,Y,NaN,442300100,365
1011000,stream gauge,USGS,"Allagash River near Allagash, Maine",1011000,https://waterservices.usgs.gov/nwis/site/?form...,2023-05-05,47.069722,-69.079444,ME,1010002,...,NaN,YNNNYNNNNYNNNNNNYNNNNNNNNNNNNN,NaN,NaN,1478.0000,1229.0,Y,NaN,442300100,365
1013500,stream gauge,USGS,"Fish River near Fort Kent, Maine",1013500,https://waterservices.usgs.gov/nwis/site/?form...,2023-05-05,47.237500,-68.582778,ME,1010003,...,NaN,YNNNYNYNNYNNNNNNYNNNNNNNNNNNNN,NaN,NaN,873.0000,873.0,Y,NaN,442300100,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424302090225601,stream gauge,USGS,"U.W. PLATTEVILLE FARMS SITE 03 NR PLATTEVILLE, WI",424302090225601,https://waterservices.usgs.gov/nwis/site/?form...,2023-05-05,42.717218,-90.382350,WI,7060005,...,NaN,NNYYNNNNNNYYNNNNNNNNNNNNNNNNNN,NaN,NaN,0.0220,NaN,Y,NaN,2491BQY13,365
424314090240601,stream gauge,USGS,"U.W. PLATTEVILLE FARMS SITE 02 NR PLATTEVILLE, WI",424314090240601,https://waterservices.usgs.gov/nwis/site/?form...,2023-05-05,42.720556,-90.401667,WI,7060005,...,NaN,NNYYNNNNNNYYNNNNNNNNNNNNNNNNNN,NaN,NaN,0.0327,NaN,Y,NaN,2491BQY13,365
430140089281000,stream gauge,USGS,"KRONCKE DRIVE STORM SEWER AT MADISON, WI",430140089281000,https://waterservices.usgs.gov/nwis/site/?form...,2023-05-05,43.027775,-89.469565,WI,7090001,...,F,NNYNNNNNNNNYNNYNYNNNNNNNNNNNNN,NaN,NaN,0.0800,NaN,Y,NaN,2491BQY29,365


In [7]:
obs_data

,2002-10-01,2002-10-02,2002-10-03,2002-10-04,2002-10-05,2002-10-06,2002-10-07,2002-10-08,2002-10-09,2002-10-10,...,2003-09-21,2003-09-22,2003-09-23,2003-09-24,2003-09-25,2003-09-26,2003-09-27,2003-09-28,2003-09-29,2003-09-30
site_id,,,,,,,,,,,,,,,,,,,,,
1010000,9.169200,7.839100,7.725900,6.792000,5.77320,5.12230,4.72610,4.47140,4.131800,3.93370,...,4.07520,3.99030,3.99030,4.38650,6.39580,7.07500,6.84860,7.47120,8.97110,9.19750
1010070,0.597130,0.506570,0.458460,0.435820,0.42733,0.39054,0.37922,0.36224,0.370730,0.39903,...,0.94805,0.89994,0.90277,1.03578,1.35274,1.42066,1.39236,1.33576,1.64423,1.52254
1010500,8.320200,11.461500,12.933100,11.376600,11.40490,10.30120,9.28240,8.20700,7.527800,7.44290,...,11.60300,11.40490,11.12190,11.91430,13.15950,13.95190,15.90460,18.05540,58.58100,74.14600
1011000,7.414600,7.386300,6.990100,6.565600,6.36750,6.19770,5.74490,5.06570,4.528000,4.41480,...,12.65010,12.36710,11.94260,12.70670,13.69720,13.61230,13.15950,14.00850,66.22200,76.12700
1013500,3.509200,3.424300,3.169600,3.113000,3.33940,2.85830,2.85830,2.63190,2.405500,2.50455,...,14.37640,13.81040,13.61230,13.55570,13.15950,12.25390,11.94260,15.76310,58.29800,51.22300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424302090225601,0.002331,0.000111,0.000000,0.014405,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
424314090240601,0.000070,0.000017,0.000000,0.007243,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
430140089281000,0.000000,0.000849,0.000283,0.009339,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [8]:
maxcol = obs_data.max(axis=1)
print(maxcol)
maxcol.max()

site_id
1010000            481.100000
1010070            109.238000
1010500            939.560000
1011000            322.620000
1013500            218.759000
                      ...    
424302090225601      0.014405
424314090240601      0.007243
430140089281000      0.021791
430209089274900      0.025470
430230089284300      0.018395
Length: 6987, dtype: float64


28583.0

# Need to finish the Obs data annual average

In [6]:
# compute annual average for observations
obs_data_sum = obs_data.sum(axis=1)
obs_data_sum

site_id
1011000            15270.340400
1013500            14383.027860
1015800            12261.456100
1017000            24401.816500
1017550               52.182936
                       ...     
14400000           25202.148990
393109104464500      105.331185
394839104570300      559.956535
401733105392404       74.228070
402114105350101      596.102427
Length: 5714, dtype: float64

In [67]:
test = np.squeeze(read_pfb(f'{directory_out}/flow.2003.daily.001.pfb'))/3600/24
test.shape

(3256, 4442)

In [68]:
test[2783,4210]

37.266463978539505

In [69]:
test[2810,4237]

18.686342036256757

In [70]:
test[2747,4276]

30.46872761299625

In [36]:
testarray = np.zeros(5714)
testarray.shape

(5714,)

In [42]:
pf_flow_matched = np.zeros(obs_data.shape)
pf_flow_matched.shape

(5714, 365)

In [ ]:
# make CONUS2 x and y into arrays for the for loop
conusy = np.asarray(metadata['conus2_y'],dtype = 'int')
conusx = np.asarray(metadata['conus2_x'],dtype = 'int')

# Set up arrays 
pf_flow_array = np.zeros((no_days, ny, nx))
pf_flow_matched = np.zeros(obs_data.shape)

### READ STREAMFLOW PFBs ###
#Read in CONUS2 daily streamflow PFBs and save as df in flow_df, convert to total accumulated in m^3/h to mean daily in cms
for i in range(no_days):
    step = str(int(i+1)).rjust(3, '0')
    print(f'{directory_out}/flow.2003.daily.{step}.pfb')
    pf_flow_pfb = np.squeeze(read_pfb(f'{directory_out}/flow.2003.daily.{step}.pfb'))
    pf_flow_array[i,...] = pf_flow_pfb/3600/24 # CONVERT FROM m^3/h to cms AND from daily accumulated to daily mean
    print(f'reading flow for day {step} and converting from total accumulated flow in m^3/h, to daily mean flow in cms')
    for j in range(len(obs_data.index)):
        if conusy[j] < 0:
            pf_flow_matched[j] = 'nan'
            #print('Gage is outside of CONUS2 range')
        else:
            pf_flow_matched[j,i] = pf_flow_array[i, conusy[j], conusx[j]]
            #print('Finding value in CONUS2 grid')

In [11]:
# Organize daily flow matched array with the same index and dates as the obs_data
pf_flow_match_df = pd.DataFrame(pf_flow_matched)
pf_flow_match_df = pf_flow_match_df.set_index(obs_data.index)
column_headers = list(obs_data.columns.values)
pf_flow_match_df.columns = column_headers

In [15]:
pf_flow_match_df_new = pf_flow_match_df.dropna()
pf_flow_match_df_new

,2002-10-01,2002-10-02,2002-10-03,2002-10-04,2002-10-05,2002-10-06,2002-10-07,2002-10-08,2002-10-09,2002-10-10,...,2003-09-21,2003-09-22,2003-09-23,2003-09-24,2003-09-25,2003-09-26,2003-09-27,2003-09-28,2003-09-29,2003-09-30
site_id,,,,,,,,,,,,,,,,,,,,,
1011000,37.266464,34.151593,31.949418,28.746042,24.794453,21.037749,17.725660,14.927488,12.823219,11.328315,...,0.383030,0.248542,0.270354,0.370566,0.447849,0.437744,0.346303,1.857882,32.173826,0.0
1013500,18.686342,17.994405,17.171830,14.470901,14.358695,12.452931,11.061765,10.040905,9.119768,8.499280,...,0.542694,0.269608,0.231789,0.451696,0.321280,0.249696,0.204132,1.203276,30.743592,0.0
1015800,30.468728,29.614383,27.459663,24.822553,22.551353,19.456863,16.647201,14.524086,12.689513,11.405875,...,0.397749,0.306602,0.316849,0.505316,0.353142,0.297726,0.204833,0.469169,6.413734,0.0
1017000,64.752051,64.284799,62.117018,57.702024,52.771326,48.076338,43.703179,39.512687,35.268489,31.090277,...,1.916867,1.738682,1.658574,1.623249,1.463889,1.393643,1.341100,1.439029,3.284965,0.0
1017550,0.044492,0.028687,0.010159,0.003003,0.004380,0.001411,0.000000,0.000000,0.000000,0.000036,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14366000,17.141487,14.177220,9.895453,7.432401,5.706502,3.889199,2.879436,1.984967,1.299827,0.876629,...,0.000858,0.001007,0.001868,0.003321,0.004422,0.001325,0.000117,0.001613,0.002378,0.0
14369500,23.485204,19.583931,15.918011,12.305456,8.867973,6.812724,4.987838,3.594443,2.754412,2.164704,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
14372300,158.534609,146.141058,138.795475,132.012636,125.932543,121.008399,112.250051,96.432045,80.411230,67.641496,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [16]:
obs_data_new = obs_data[obs_data.index.isin(pf_flow_match_df_new.index)]
obs_data_new

,2002-10-01,2002-10-02,2002-10-03,2002-10-04,2002-10-05,2002-10-06,2002-10-07,2002-10-08,2002-10-09,2002-10-10,...,2003-09-21,2003-09-22,2003-09-23,2003-09-24,2003-09-25,2003-09-26,2003-09-27,2003-09-28,2003-09-29,2003-09-30
site_id,,,,,,,,,,,,,,,,,,,,,
1011000,7.414600,7.386300,6.990100,6.565600,6.36750,6.19770,5.744900,5.065700,4.528000,4.414800,...,12.650100,12.367100,11.942600,12.70670,13.697200,13.612300,13.159500,14.008500,66.22200,76.127000
1013500,3.509200,3.424300,3.169600,3.113000,3.33940,2.85830,2.858300,2.631900,2.405500,2.504550,...,14.376400,13.810400,13.612300,13.55570,13.159500,12.253900,11.942600,15.763100,58.29800,51.223000
1015800,6.707100,6.226000,5.716600,5.377000,5.15060,5.03740,4.754400,4.499700,4.160100,4.188400,...,3.763900,3.650700,3.537500,3.59410,3.735600,3.792200,3.707300,4.216700,25.41340,60.845000
1017000,9.876700,9.367300,8.461700,7.952300,7.81080,7.13160,7.046700,6.678800,6.282600,6.282600,...,7.075000,6.905200,6.792000,6.82030,6.820300,6.848600,6.763700,7.895700,50.65700,147.726000
1017550,0.023772,0.024338,0.020659,0.018395,0.01981,0.01981,0.018678,0.018678,0.018961,0.021791,...,0.080655,0.043016,0.049525,0.07358,0.053204,0.026319,0.023206,0.063675,0.11320,0.105559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14366000,6.509000,6.537300,6.509000,6.707100,6.56560,6.59390,6.650500,6.593900,6.565600,6.593900,...,7.414600,7.386300,7.301400,7.30140,7.301400,7.329700,7.358000,7.301400,7.30140,7.358000
14369500,6.339200,6.197700,6.226000,6.424100,6.56560,6.45240,6.452400,6.452400,6.452400,6.735400,...,7.159900,7.442900,7.273100,6.90520,6.707100,6.509000,6.565600,6.480700,6.48070,6.480700
14372300,37.922000,39.903000,41.601000,41.318000,41.88400,42.45000,40.752000,40.186000,45.280000,52.072000,...,42.450000,39.620000,38.771000,37.92200,37.073000,36.507000,36.224000,36.507000,36.79000,37.639000


In [18]:
metadata_new = metadata[metadata.index.isin(pf_flow_match_df_new.index)]
metadata_new

,site_type,agency,site_name,site_id_agency,site_query_url,date_metadata_last_updated,latitude,longitude,state,huc,...,topo_cd,instruments_cd,construction_dt,inventory_dt,drain_area_va,contrib_drain_area_va,local_time_fg,reliability_cd,project_no,num_obs
site_id,,,,,,,,,,,,,,,,,,,,,
1011000,stream gauge,USGS,"Allagash River near Allagash, Maine",1011000,https://waterservices.usgs.gov/nwis/site/?form...,2022-08-30,47.069722,-69.079444,ME,1010002,...,NaN,YNNNYNNNNYNNNNNNYNNNNNNNNNNNNN,NaN,NaN,1478.00,1229.00,Y,NaN,442300100.0,365
1013500,stream gauge,USGS,"Fish River near Fort Kent, Maine",1013500,https://waterservices.usgs.gov/nwis/site/?form...,2022-08-30,47.237500,-68.582778,ME,1010003,...,NaN,YNNNYNYNNYNNNNNNYNNNNNNNNNNNNN,NaN,NaN,873.00,873.00,Y,NaN,442300100.0,365
1015800,stream gauge,USGS,"Aroostook River near Masardis, Maine",1015800,https://waterservices.usgs.gov/nwis/site/?form...,2022-08-30,46.523056,-68.371667,ME,1010004,...,NaN,YNNNYNYNNYNNNNNNYNNNNNNNNNNNNN,NaN,NaN,892.00,892.00,Y,NaN,442300100.0,365
1017000,stream gauge,USGS,"Aroostook River at Washburn, Maine",1017000,https://waterservices.usgs.gov/nwis/site/?form...,2022-08-30,46.777222,-68.157222,ME,1010004,...,NaN,YNNNYNYNNYNNNNNNYNNNNNNNNNNNNN,NaN,NaN,1654.00,1654.00,Y,NaN,442300100.0,365
1017550,stream gauge,USGS,"Williams Brook at Phair, Maine",1017550,https://waterservices.usgs.gov/nwis/site/?form...,2022-08-30,46.628056,-67.953056,ME,1010005,...,NaN,YNNNYNYNNYNNNNNNYNNNNNNNNNNNNN,NaN,NaN,3.82,3.82,Y,NaN,442300100.0,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14366000,stream gauge,USGS,"APPLEGATE RIVER NEAR APPLEGATE, OR",14366000,https://waterservices.usgs.gov/nwis/site/?form...,2022-08-30,42.241511,-123.140049,OR,17100309,...,NaN,NNNNYNNNNNNNNNNNNNNNNNNNNNNNNN,NaN,NaN,483.00,NaN,Y,C,NaN,365
14369500,stream gauge,USGS,"APPLEGATE RIVER NEAR WILDERVILLE, OR",14369500,https://waterservices.usgs.gov/nwis/site/?form...,2022-08-30,42.354006,-123.406728,OR,17100309,...,NaN,NNNNYNNNNNNNNNNNNNNNNNNNNNNNNN,NaN,NaN,698.00,NaN,Y,C,NaN,365
14372300,stream gauge,USGS,"ROGUE RIVER NEAR AGNESS, OR",14372300,https://waterservices.usgs.gov/nwis/site/?form...,2022-08-30,42.578442,-124.058144,OR,17100310,...,NaN,NNNNYNNNNNNNNNNNNNNNNNNNNNNNNN,NaN,NaN,3939.00,NaN,Y,C,NaN,365


In [20]:
# save matched daily average flow and metadata as csv's
pf_flow_match_df_new.to_csv(f'{organized_dir}/PFCONUS2_Daily_matched_flow_cms_{water_year}.csv', sep = ",")
obs_data_new.to_csv(f'{organized_dir}/USGS_Daily_matched_flow_cms_{water_year}.csv', sep = ",")
metadata_new.to_csv(f'{organized_dir}/USGS_metadata_matched_flow_{water_year}.csv', sep = ",")

In [ ]:
# # Calculate Annual Average Flow for PFCLM and save
# pf_flow_avg = np.mean(pf_flow_array, axis = 0) #monthly average 
# pf_flow_avg.shape
# np.savetxt(f'{organized_dir}/PFCONUS2_Annual_Avg_Flow_cms_WY2003.csv', pf_flow_avg, delimiter=",")

In [ ]:
pf_flow_array.shape

In [ ]:
pf_flow_match.shape